Referance: https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb

In [1]:
%pip install -q amplpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 9.0 MB/s eta 0:00:00


In [2]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gurobi", "coin", "highs", "gokestrel"],  # modules to install
    license_uuid="default",  # license to use
)

# Problem

Given $n$ items with weights $w_{i}$ for all $i \in \{1,...,n\}$ and bins with capacity $C$ (for each bin), the bin packing problem assigns each item $i$ to a bin while minimizing the number of used bins. BPP can be modeled as follows:

$$
\begin{aligned}
\text{minimize} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{subject to} \quad &\sum_{i = 1}^{n} w_{i} x_{i j} \leq C y_{j} \quad \forall j \in \{1,\ldots, m\} \\
&\sum_{j = 1}^{m} x_{i j} = 1 \quad \forall i \in \{1,\ldots,n\} \\
&x_{i j}, y_j \in \{0,1\} \quad \forall i \in \{1,\ldots,n\}, j \in \{1,\ldots,m\}
\end{aligned}
$$

# Determine a proper range of the number of bins.

* A lower bound of $m$ can be found from $\sum w_i \leq C*m$, but the inequality is already included in the first two conditions above. Moreover, what we need is an upper bound of $m$ which will be used as a parameter in the AMPL model.

* Let $W$ be the maximum weight. To find an upper bound of $m$, we can suppose that we want to assign each of $n$ items with all weights $W$ to the bins. The number of items each bin    

In [3]:
%%ampl_eval
param n;
param C;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param maxVal := max {i in I} w[i];
param maxbins := ceil(n / floor(C / maxVal));

set J = 1..maxbins;

var x {I,J} binary;
var y {J} binary;

minimize Cost:  sum {j in J} y[j];

subject to b_Capacity {j in J}:
   sum {i in I} w[i] * x[i,j] <= C * y[j] suffix block j;

subject to m_Allocate {i in I}:
   sum {j in J} x[i,j] >= 1 suffix master 1;


If bins have different capacities? Objective can be different, say, minimize the number of bins or minimize the total capacities of bins used...